In [1]:
import os
from dotenv import load_dotenv
import json
import requests
from pymongo import MongoClient

# Load environment variables from .env file
load_dotenv()

# Set up connection to MongoDB using environment variables
mongodb_url = os.environ.get("mongodb_url")
db_name = os.environ.get("DB_NAME")
collection_name = os.environ.get("COLLECTION_NAME")

client = MongoClient(mongodb_url)
db = client[db_name]
collection = db[collection_name]

# Read the quotes from JSON file
with open("quotes.json") as f:
    quotes_data = json.load(f)["quotes"]

# Define function to add quotes to database
def add_text(author, text, img_url=None, video_url=None):
    existing_text = collection.find_one({"author": author, "text": text})
    if existing_text:
        print("Quote already exists in the database.")
        return
    new_text = {
        "author": author,
        "text": text
    }
    if img_url:
        new_text["img_url"] = img_url
    if video_url:
        new_text["video_url"] = video_url
    collection.insert_one(new_text)
    print("Quote added successfully.")

# Define function to send text to API
def send_text_to_api(author, text, img_url=None, video_url=None):
    url = os.environ.get("API_URL")
    headers = {"Content-Type": "application/json"}
    data = {"author": author, "text": text}
    if img_url:
        data["imgUrl"] = img_url
    if video_url:
        data["videoUrl"] = video_url

    try:
        response = requests.post(url, json=data, headers=headers)
        response.raise_for_status()
        print(response.json())
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except json.JSONDecodeError as e:
        print(f"JSON decode error occurred: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Add the texts to the database and send them to the API
for quote in quotes_data:
    author = quote["author"]
    text = quote["text"]
    img_url = quote.get("imgUrl")
    video_url = quote.get("videoUrl")
    add_text(author, text, img_url, video_url)
    send_text_to_api(author, text, img_url, video_url)
